In [4]:
import numpy as np

def one_qubit_readout_error_matrix(qubit_n, scale = 1):
    """
    Input: which qubit on the london device, scale (for noise scaling)
    Generates the readout error matrix
    
    organized as 
    
    P(0|0)  P(0|1)
    P(1|0)  P(1|1)
    
    """
    if (qubit_n == 0): 
        errors = [0.04, 0.05]
    elif (qubit_n == 1): 
        errors = [0.06, 0.07]
        
    x, y = errors
    x = scale*x
    y = scale*y
    
    matrix = [[1-x, y], [x, 1-y]]
    
    return matrix
    

In [7]:
def column_sum_to_1(matrix): 
    """
    Makes sure that columns of a 2x2 matrix sum to 1
    """
    matrix = np.transpose(matrix)
    
    for r in range(len(matrix)):
        summ = 0
        for c in range(len(matrix[r])):
            summ += matrix[r][c]
        for c in range(len(matrix[r])): 
            matrix[r][c] = (matrix[r][c])/summ
            
    matrix = np.transpose(matrix)
    return matrix
        

def two_qubit_readout_error_matrix(mat_1, mat_2):
    """
    Input: corresponding readout error matrices of the 2 qubits
    Output: their kronecker product
    """
    P = mat_1
    Q = mat_2
    
    matrix = column_sum_to_1(np.kron(P,Q))
    return matrix


In [8]:
mat = [[0.7, 0.9],[0.6, 1.8]]
column_sum_to_1(mat)

array([[0.53846154, 0.33333333],
       [0.46153846, 0.66666667]])

In [11]:
def get_2_qubit_readout_matrix_regular(scale): 
    """
    for amplifying readout errors
    """
    
    #what should happen after all
    P_scale =  one_qubit_readout_error_matrix(0, scale)
    Q_scale = one_qubit_readout_error_matrix(1, scale)
    R_scale = two_qubit_readout_error_matrix(P_scale, Q_scale)
    
    return R_scale

def get_2_qubit_readout_matrix_after_readout_measurement(scale): 
    """
    for amplifying readout errors
    """
    #what already happened
    P_1 = one_qubit_readout_error_matrix(0)
    Q_1 = one_qubit_readout_error_matrix(1)
    R_1 = two_qubit_readout_error_matrix(P_1, Q_1)
    
    #what should happen after all
    P_scale =  one_qubit_readout_error_matrix(0, scale)
    Q_scale = one_qubit_readout_error_matrix(1, scale)
    R_scale = two_qubit_readout_error_matrix(P_scale, Q_scale)
    
    #what then needs to happen 
    """
    XR_1 = R_scale
    X = R_scale*(R_1)^-1
    """
    inverse = np.linalg.inv(R_1)
    X = np.dot(R_scale, inverse)
    
    return X
    

In [10]:
get_2_qubit_readout_matrix_after_readout_measurement(4)

array([[0.68851838, 0.20954907, 0.13073134, 0.0397878 ],
       [0.17961349, 0.6585828 , 0.03410383, 0.12504737],
       [0.10458507, 0.03183024, 0.66237211, 0.20159151],
       [0.02728306, 0.10003789, 0.17279272, 0.63357332]])

In [14]:
get_2_qubit_readout_matrix_after_readout_measurement(5).dot(get_2_qubit_readout_matrix_regular(1))

array([[0.56  , 0.28  , 0.175 , 0.0875],
       [0.24  , 0.52  , 0.075 , 0.1625],
       [0.14  , 0.07  , 0.525 , 0.2625],
       [0.06  , 0.13  , 0.225 , 0.4875]])

In [13]:
get_2_qubit_readout_matrix_regular(5)

array([[0.56  , 0.28  , 0.175 , 0.0875],
       [0.24  , 0.52  , 0.075 , 0.1625],
       [0.14  , 0.07  , 0.525 , 0.2625],
       [0.06  , 0.13  , 0.225 , 0.4875]])

In [23]:
def counts_to_array(counts): 
    """
    Turning counts (dict) to array format ...for matrix multiplication
    """
    array = [[counts['00']],[counts['01']], [counts['10']], [counts['10']] ]
    return array
def array_to_counts(array, shots = 1024):
    """
    Turning array into counts(dict)
    """
    #making sure the array has correct number of shots
    summ = 0
    for i in array: 
        summ+=i
    new_array = [(i/summ)*1024 for i in array]
    
    counts = {'00': float(new_array[0]), '01': float(new_array[1]), '10':float(new_array[2]), '11': float(new_array[3])}
    
    
    return counts

In [24]:
def counts_after_readout_amp(counts, scale):
    """
    Input: Counts, amplification factor
    Output: modified counts
    """
    array = counts_to_array(counts)
    matrix = get_2_qubit_readout_matrix_after_readout_measurement(scale)
    new_array = np.dot(matrix, array)
    
    return array_to_counts(new_array)
    
    

In [26]:
counts = {'00': 100, '01': 0, '10': 0, '11': 0}
counts_after_readout_amp(counts, 2)

{'00': 911.4725274725275,
 '01': 67.5164835164835,
 '10': 41.90678287230012,
 '11': 3.104206138688896}